# Incremental Dynamic Learning

## Overview

The idea behind Incremental Dynamic Learning networks is that information is not always learned at once, up front. As information is encountered, we train on the new data. Adding the new information to our existing knowlege base. Humans, are said to be able to recognize an image in 13 milliseconds after seeing it.The new image is added to their existing learned information and they do not have to retrain on all images they have ever seen.

## Example

As an example of how this can be used in the real world, I will use field biologists for my use case.

Imagine a group of field biologists are in the field and all have a mobile application which allows them to photograph wildlife they encounter, label it and forward it to a central system. The central system takes in the new photographs and trains a neural network on each piece of new data that comes in. W/out having to retrain a new model based on all past photographs. 